In [1]:
import os
os.chdir("../")

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    epochs: int
    warmup_steps: int
    train_batch_size_per_device: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    evaluation_steps: int
    save_steps: int
    gradient_accumulation_steps: int


In [3]:
from textsummarizer.constants import *
from textsummarizer.utils.common_func import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_file_path = CONFIG_FILE_PATH, params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainingConfig:
        config = self.config.model_training
        params = self.params.TrainingArgs

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainingConfig(
            root_dir=Path(config.root_dir),
            data_path= config.data_path,
            model_ckpt= config.model_ckpt,
            epochs=params.epochs,
            warmup_steps= params.warmup_steps,
            train_batch_size_per_device= params.train_batch_size_per_device,
            weight_decay=params.weight_decay,
            logging_steps= params.logging_steps,
            evaluation_strategy= params.evaluation_strategy,
            evaluation_steps = params.evaluation_steps,
            save_steps= params.save_steps,
            gradient_accumulation_steps= params.gradient_accumulation_steps

        )

        return model_trainer_config

In [ ]:
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

class ModelTrainer:
    def __init__(self, config: ModelTrainingConfig):
        self.config = config

    def train_model(self):
        my_device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"Using device: {my_device}")
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        pegasus_model = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(my_device)
        seg2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=pegasus_model)

        dataset = load_from_disk(self.config.data_path)

        training_arguments = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=self.config.epochs,
            per_device_train_batch_size=self.config.train_batch_size_per_device,
            warmup_steps=self.config.warmup_steps,
            weight_decay=self.config.weight_decay,
            logging_steps=self.config.logging_steps,
            do_eval=True,
            fp16=True,
            save_steps=self.config.save_steps,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps,
        )

        trainer = Trainer(
            model=pegasus_model.to(my_device),
            args=training_arguments,
            data_collator=seg2seq_data_collator,
            train_dataset=dataset["train"],
            eval_dataset=dataset["validation"],
        )

        trainer.train()

        pegasus_model.save_pretrained(os.path.join(self.config.root_dir, "pegasus_model_trained"))

        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "pegasus_model_tokenizer"))


        


In [9]:

PYTORCH_MPS_HIGH_WATERMARK_RATIO = 0.0

In [10]:
try:
    config = ConfigurationManager()
    config_data_transformation = config.get_model_trainer_config()
    data_transformation = ModelTrainer(config = config_data_transformation)
    data_transformation.train_model()
except Exception as e:
    raise e

[2025-06-22 15:19:57,357: INFO:  common_func:  Yaml file: config/config.yaml has been loaded succesfully]
[2025-06-22 15:19:57,361: INFO:  common_func:  Yaml file: params.yaml has been loaded succesfully]
[2025-06-22 15:19:57,362: INFO:  common_func:  created directory at: artifacts]
[2025-06-22 15:19:57,364: INFO:  common_func:  created directory at: artifacts/model_training]
Using device: cuda


Step,Training Loss
10,3.336000
20,3.367800
30,3.280800
40,3.269600
50,3.118500
60,3.131900
70,3.067800
80,3.015700
90,2.903000
100,2.869300


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/sachin/miniconda3/envs/myenv/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3579, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_23466/1359606082.py", line 7, in <module>
    raise e
  File "/tmp/ipykernel_23466/1359606082.py", line 5, in <module>
    data_transformation.train_model()
  File "/tmp/ipykernel_23466/3275576084.py", line 43, in train_model
    pegasus_model.save_model(os.path.join(self.config.root_dir, "pegasus_model_trained"))
  File "/home/sachin/miniconda3/envs/myenv/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1940, in __getattr__
    raise AttributeError(
AttributeError: 'T5ForConditionalGeneration' object has no attribute 'save_model'. Did you mean: 'base_model'?

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/sachin/miniconda3/envs/myenv/lib/python3.10/site-packages/I

In [7]:
import torch
torch.cuda.empty_cache()
